<a href="https://colab.research.google.com/github/Naaastenka/Project1/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Реализовать задание 1 и 2 используя объектно-ориентированный подход в python. Справочно:

Задание 1 Собрать датасет для исследования по данным об объектах недвижимости (строящихся/готовых) на сайте наш.дом.рф или cian.ru При использовании сервиса наш.дом.рф возможно появлении ошибки JSONDecodeError. В этом случае используйте cian.ru Полученные данные нужно преобразовать в pandas DataFrame и сохранить его в разных вариантах: 1. сsv/ xlsx 2. pickle 3. база данных Датасет будет использован для последующих заданий поэтому важно, чтобы он содержал достаточное количество наблюдений(желательно не менее 1000 записей)

Задание 2 по визуализации на python * Используя данные об объектах недвижимости на сайте "наш.дом.рф" или cian.ru, провести мини-исследование рынка строящейся/готовой недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов. Идеи для мини-исследования * Рассчитать средние цены за квадратный метр по регионам. Важно, чтобы пропуски не учитывались при расчете средней цены. * Рассчитать объемы вводимого жилья по регионам и по годам. * Сравнить этажность домов, количество квартир, жилую площадь, количество парковочных мест и цену за квадратный метр в разных регионах. * Визуализировать разброс, связь, доли, распределение этих показателей по регионам. * Сделать выводы по каждому из расчетов и графиков

In [1]:
pip install git+https://github.com/lenarsaitov/cianparser.git

  Cloning https://github.com/lenarsaitov/cianparser.git to /tmp/pip-req-build-bcb2_o9q
  Running command git clone --filter=blob:none --quiet https://github.com/lenarsaitov/cianparser.git /tmp/pip-req-build-bcb2_o9q
  Resolved https://github.com/lenarsaitov/cianparser.git to commit 236352a200b08b27c24d14b18939b4c34691cbb2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 12.8 MB/s eta 0:00:00
  Created wheel for cianparser: filename=cianparser-1.0.4-py3-none-any.whl size=32586 sha256=6b800ff1290b3a47902ccdce73b3ace346f0054fcca1b644e1972c2cda307bb4
  St

In [2]:
pip install pandas sqlalchemy openpyxl

In [3]:
import cianparser
import pandas as pd
import pickle
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
class RealEstateDataProcessor:
    def __init__(self, location):
        self.parser = cianparser.CianParser(location=location)
        self.df = None

    def collect_data(self, deal_type, rooms, start_page, end_page):
        data = self.parser.get_flats(deal_type=deal_type, rooms=rooms, with_saving_csv=False, additional_settings={"start_page": start_page, "end_page": end_page})
        self.df = pd.DataFrame(data)
        print(self.df.head())
        print(self.df.info())

    def save_to_csv(self, file_path):
        self.df.to_csv(file_path, index=False)

    def save_to_excel(self, file_path):
        self.df.to_excel(file_path, index=False)

    def save_to_pickle(self, file_path):
        with open(file_path, 'wb') as f:
            pickle.dump(self.df, f)

    def save_to_sqlite(self, db_path):
        engine = create_engine(f'sqlite:///{db_path}')
        self.df.to_sql('real_estate', engine, index=False, if_exists='replace')

    def clean_data(self):
        self.df = self.df.dropna(subset=['price', 'square_meter_price', 'region', 'year_built'])
        self.df['price'] = self.df['price'].astype(float)
        self.df['square_meter_price'] = self.df['square_meter_price'].astype(float)
        self.df['year_built'] = self.df['year_built'].astype(int)

    def visualize_data(self):
        mean_price_per_sqm = self.df.groupby('region')['square_meter_price'].mean().reset_index()
        plt.figure(figsize=(10, 6))
        sns.barplot(x='square_meter_price', y='region', data=mean_price_per_sqm, palette='viridis')
        plt.xlabel('Средняя цена за квадратный метр')
        plt.ylabel('Регион')
        plt.title('Средняя цена за квадратный метр по регионам')
        plt.show()

        volume_by_region_year = self.df.groupby(['region', 'year_built']).size().reset_index(name='count')
        plt.figure(figsize=(12, 8))
        sns.lineplot(x='year_built', y='count', hue='region', data=volume_by_region_year, palette='tab10')
        plt.xlabel('Год')
        plt.ylabel('Объем вводимого жилья')
        plt.title('Объемы вводимого жилья по регионам и по годам')
        plt.legend(title='Регион')
        plt.show()

        metrics = ['floors', 'number_of_flats', 'living_area', 'parking_spaces', 'square_meter_price']
        for metric in metrics:
            plt.figure(figsize=(10, 6))
            sns.boxplot(x='region', y=metric, data=self.df, palette='muted')
            plt.xlabel('Регион')
            plt.ylabel(metric)
            plt.title(f'{metric} по регионам')
            plt.xticks(rotation=90)
            plt.show()


Пример использования класса

In [ ]:
processor = RealEstateDataProcessor(location="Москва")
processor.collect_data(deal_type="sale", rooms=(1, 2), start_page=1, end_page=20)
processor.save_to_csv('data.csv')
processor.save_to_excel('data.xlsx')
processor.save_to_pickle('data.pkl')
processor.save_to_sqlite('data.db')
processor.clean_data()
processor.visualize_data()
print("Все данные сохранены и визуализированы")


                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&with_neighbors=0&region=1&deal_type=sale&offer_type=flat&room1=1&room2=1












Exception: 429 Client Error: Too Many Requests for url: https://www.cian.ru/cat.php?engine_version=2&p=11&with_neighbors=0&region=1&deal_type=sale&offer_type=flat&room1=1&room2=1
The collection of information from the pages with ending parse on 11 page...




Exception: 429 Client Error: Too Many Requests for url: https://www.cian.ru/cat.php?engine_version=2&p=13&with_neighbors=0&region=1&deal_type=sale&offer_type=flat&room1=1&room2=1
The collection of information from the pages with ending parse on 13 page...



Exception: 429 Client Error: Too Many Requests for url: https://www.cian.ru/cat.php?engine_version=2&p=14&with_neighbors=0&region=1&deal_type=sale&offer_type=flat&room1=1&room2=1
The collection of information from 